In [1]:
# RAG with Gemini Flash 1.5 LLM and ARES evaluation
# Google Gemini: https://ai.google.dev/gemini-api/docs/models/gemini
# ARES: https://github.com/stanford-futuredata/ARES

In [2]:
# Establish RAG pipeline with Gemini

In [1]:
import os
from IPython.display import display, Markdown
import pandas as pd

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext, load_index_from_storage
from llama_index.vector_stores.faiss import FaissVectorStore
#from llama_index.llms.gemini import Gemini
#from llama_index.embeddings.gemini import GeminiEmbedding

import faiss

In [2]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
# set up local API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [5]:
 # old function

In [5]:
# Create a Faiss vector store for RAG
# Example of creating a small vector store
# Using 4 State of the Union speeches, all text from whitehouse.gov briefing room speeches posted online, edited to include a title with the date of the speech
# Example from 2024:
# https://www.whitehouse.gov/briefing-room/speeches-remarks/2024/03/07/remarks-of-president-joe-biden-state-of-the-union-address-as-prepared-for-delivery-2/

# load and parse files
sotu = []
newfiles = ["./Speeches/titleedits/state_of_the_union_042921.txt", "./Speeches/titleedits/state_of_the_union_030122.txt", "./Speeches/titleedits/state_of_the_union_020723.txt", "./Speeches/titleedits/state_of_the_union_030724.txt"]
for i in newfiles:
    with open(i) as file:
        for line in file:
            nl = line.rstrip()
            if nl != '':
                sotu.append(nl)

# convert into Document format
documents = [Document(text=line) for line in sotu]

In [7]:
# Example of a loaded Document line
documents[-1]

Document(id_='4a26868c-5225-4612-9338-005a366b96e5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='May God protect our troops.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [8]:
# Set up the faiss index
d = 768 # dimensions of the input vector of the embedding model that we're going to use; in this case, the google embedding model
faiss_index = faiss.IndexFlatL2(d)
print(faiss_index.is_trained) # double check that the training worked

True


In [6]:
# Set up the llm, embeddings, and Settings for the llamaIndex FaissVectorStore
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
doc_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") # optional: task_type="RETRIEVAL_DOCUMENT"
Settings.embed_model = doc_embeddings
Settings.llm = llm

In [12]:
# # Uncomment for when you need to re-embed and vectorize documents
# # Otherwise, doing local loading below

# vector_store = FaissVectorStore(faiss_index=faiss_index)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, show_progress=True
# )

# # Save index to disk
# index.storage_context.persist()

# # Save/remember index id for loading next time
# index.index_id

In [7]:
# After you have a saved index, load that index for RAG answer generation:

# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
# My local index id '3d3c99c5-aa1c-42d7-a9ce-c4bb12fbc6d5' uses the 4 speeches including a title that includes the date it was given
index = load_index_from_storage(storage_context=storage_context, index_id='3d3c99c5-aa1c-42d7-a9ce-c4bb12fbc6d5')

In [10]:
# Set up query and chat engines with the index
query_engine = index.as_query_engine(similarity_top_k=10)
chat_engine = index.as_chat_engine(similarity_top_k=10, chat_mode='context')

In [13]:
# Example query and response with Gemini and query_engine
# query = "What does the President say about his administration's first 100 days and covid-19?"
# response = query_engine.query(query) 
# print(response.response)

In [21]:
# Code for ARES RAG evaluation library to work with Gemini and our local RAG setup

In [13]:
from ares import ARES

vLLM not imported.
vLLM not imported.
vLLM not imported.
vLLM not imported.


In [16]:
# This configuration runs an evaluation on an unlabeled evaluation set (UES) in conjunction with in-domain prompts (IDP) 
ues_idp_config = {
    "in_domain_prompts_dataset": "ARES_files/nq_few_shot_prompt_for_judge_scoring.tsv", # Small set of labeled few-shot examples for scoring context relevance, answer faithfulness, and/or answer relevance in your RAG system
    "unlabeled_evaluation_set": "ARES_files/nq_unlabeled_output.tsv", # Larger set of unlabeled query-document-answer triples output by your RAG system for scoring
    "model_choice" : "models/gemini-1.5-flash",
    "request_delay" : 60, # Delay (in seconds) between requests to the API
    "documents" : 0 # Number of documents to be evaluated. Default is 0, which means all documents in the evaluation set will be evaluated
} 

ares = ARES(ues_idp=ues_idp_config)

In [15]:
results = ares.ues_idp()
print(results)

Evaluating large subset with models/gemini-1.5-flash:   0%|          | 0/3 [00:00<?, ?it/s]

configured gemini for context relevance
Testing candidates
test_response
[[Yes]]

configured gemini for answer relevance
Testing response.text
[[Yes]]

configured gemini for answer faithfulness
Testing response.text
[[Yes]]

configured gemini for context relevance
Testing candidates
test_response
[[Yes]]

configured gemini for answer relevance
Testing response.text
[[Yes]]

configured gemini for answer faithfulness
Testing response.text
[[Yes]]

configured gemini for context relevance
Testing candidates
Attempt 1 failed with error: list index out of range
Testing candidates
Attempt 2 failed with error: list index out of range
Testing candidates
Attempt 3 failed with error: list index out of range
Testing candidates
Attempt 4 failed with error: list index out of range
Testing candidates
All attempts failed. Last error was: list index out of range
Number of times did not extract Yes or No: 0
{'Context Relevance Scores': 0.667, 'Answer Faithfulness Scores': 0.667, 'Answer Relevance Scores

In [18]:
results

{'Context Relevance Scores': 0.667,
 'Answer Faithfulness Scores': 0.667,
 'Answer Relevance Scores': 0.667}

In [14]:
# According to ARES, 'document' refers to a line in a document_filepath_file
# I never tested it with the speeches broken up line by line/by paragraph/by sentence
# This setup of 1 large document per line did not work well; recommend trying documents being only a few sentences (or so) each
synth_config = { 
    "document_filepaths": ["Speeches/titleedits/Speeches_Docs_ARES.tsv"], # Source documents for generating synthetic queries; requires tsv file
    "few_shot_prompt_filename": "datasets/manual_dataset_complete_ares_synthetic.tsv", # Few shot labeled training data with queries and answers
    "synthetic_queries_filenames": ["results/synthetic_results.tsv"], # Where to save resulting synthetic data file
    "model_choice": "models/gemini-1.5-flash", 
    "documents_sampled": 1000, # How many documents to sample when generating synthetic queries; ARES filters documents < 50 words
    "api_model": True
}

ares_synth = ARES(synthetic_query_generator=synth_config)

In [15]:
results = ares_synth.generate_synthetic_data() # stopped here, need to debug/edit to work with Gemini; next is embeddings error below
print(results)

Saving synthetic queries to:  ['results/synthetic_results.tsv']

| Starting Synthetic Query Generation |

Generating positive queries for the first 2 documents...


Generating positive synthetic queries for documents 0 to 2...:   0%|                              | 0/4 [00:00<?, ?it/s]

configured gemini for synthetic query generation


Generating positive synthetic queries for documents 0 to 2...:  25%|█████▌                | 1/4 [00:06<00:18,  6.09s/it]

configured gemini for synthetic query generation
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).


...

configured gemini for synthetic answer generation


Generating answers... (models/gemini-1.5-flash): 100%|████████████████████████████████████| 2/2 [00:01<00:00,  1.69it/s]

configured gemini for synthetic answer generation


Generating answers... (models/gemini-1.5-flash): 100%|████████████████████████████████████| 2/2 [00:02<00:00,  1.27s/it]

Generated answers for 2 queries.
Saved positive answers to: results/synthetic_results.tsv
Generating negative answers for the second chunk of queries...
Saved answers to: results/synthetic_results.tsv
Completed synthetic generation!
Saved synthetic queries file to: results/synthetic_results.tsv

| Answer generation and processing completed. |

None


In [1]:
# todo: try out ARES ues idp config with my dataset/RAG
# try out ARES with their testsets

In [17]:
ues_idp_config = {
    "in_domain_prompts_dataset": "ARES_files/nq_few_shot_prompt_for_judge_scoring.tsv",
    "unlabeled_evaluation_set": "ARES_files/nq_unlabeled_output.tsv", 
    "model_choice" : "models/gemini-1.5-flash",
    "request_delay" : 60,
    "documents" : 3
} 

ares = ARES(ues_idp=ues_idp_config)

In [ ]:
results = ares.ues_idp()
print(results)

In [14]:
# with next run will confirm if PPI results are None or not/how often
ppi_config = { 
    "evaluation_datasets": ['datasets/unlabeled_dataset/unlabeled_dataset.tsv'],#['ARES_files/nq_unlabeled_output_100examples.tsv'], 
    "few_shot_examples_filepath": "datasets/manual_dataset_complete_ares.tsv", # "ARES_files/nq_few_shot_prompt_for_judge_scoring.tsv",
    "model_choice": "models/gemini-1.5-flash",
    #"num_trials" : 10, # default is 1000, just testing to see if it'll work
    "llm_judge": "models/gemini-1.5-flash",
    "labels": ["Context_Relevance_Label"], 
    "gold_label_paths": ["datasets/labeled_dataset/labeled_dataset_sotu.tsv"]#["ARES_files/nq_labeled_output.tsv"] # need 50+ examples different than few shot
    #"checkpoints": ["None"] #["/home/amber/ARES/checkpoints/ares_context_relevance_general_checkpoint_V1.1.pt"]
}

ares_ppi = ARES(ppi=ppi_config)

In [15]:
results = ares_ppi.evaluate_RAG()
print(results)

--------------------------------------------------------
Evaluation Sets: ['datasets/unlabeled_dataset/unlabeled_dataset.tsv']
Checkpoints: False
Labels: ['Context_Relevance_Label']
--------------------------------------------------------
configured gemini api key for LLMJudge


Using the latest cached version of the module from /home/amber/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Thu Oct 24 13:59:57 2024) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


Performing Model scoring!
context_relevance_label and gemini working


  0%|                                                                                           | 0/800 [00:00<?, ?it/s]

configured gemini for context relevance


...

Evaluating:  98%|█████████████████████████████████████████████████████████████████████▊ | 58/59 [03:40<00:01,  1.12s/it]

configured gemini for context relevance


Sample Ground Truth Labels: [1 1 0 1 0 1 1 1 1 1]
Sample LLM Judge Predictions: [1 0 0 1 1 1 1 1 1 1]


Trials:   0%|                                                                                  | 0/1000 [00:00<?, ?it/s]/home/amber/newARES/ARES/ares/RAG_Automatic_Evaluation/ppi.py:81: RuntimeWarning: Mean of empty slice.
  rechat = (Yhat_labeled - Y_labeled).mean()
/home/amber/anaconda3/envs/researchcopy/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/amber/anaconda3/envs/researchcopy/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/amber/anaconda3/envs/researchcopy/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/amber/anaconda3/envs/researchcopy/lib/python3.11/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value en

Calculated Accuracy: 0.6949152542372882
--------------------------------------------------
Context_Relevance_Label Scoring
ARES Ranking
Evaluation_Set: datasets/unlabeled_dataset/unlabeled_dataset.tsv
Checkpoint: None
ARES Prediction: 0.7324999999999967
ARES Confidence Interval: [0.588, 0.877]
Number of Examples in Evaluation Set: 800
ARES LLM Judge Accuracy on Ground Truth Labels: 0.695
Annotated Examples used for PPI: 59
--------------------------------------------------

[{'Label_Column': 'Context_Relevance_Label', 'Evaluation_Set': 'datasets/unlabeled_dataset/unlabeled_dataset.tsv', 'ARES_Prediction': 0.7324999999999967, 'ARES_Confidence_Interval': [0.588, 0.877], 'Number_of_Examples_in_Evaluation_Set': 800, 'Ground_Truth_Performance': None, 'ARES_LLM_Judge_Accuracy_on_Ground_Truth_Labels': 0.695, 'Annotated_Examples_used_for_PPI': 59}]
